## 전체 데이터 -> 일자별 데이터로 변환(성별,연령,국적통합)

In [ ]:
import pandas as pd
from glob import glob
import os

# 파일 경로와 패턴
base_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/결측값 제거 데이터/'  # 원본 데이터의 경로로 수정
output_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/성별,연령,국적 통합 데이터/'  # 저장할 경로로 수정

# 결과 저장 경로 확인 및 생성
os.makedirs(output_path, exist_ok=True)

# 파일 목록
csv_files = glob(base_path + 'KSR_SJUNIV_TRIP_YYD_0000000000*_cleaned.csv')

# 전체 데이터를 결합할 데이터프레임
combined_data = pd.DataFrame()

# 각 파일을 처리하여 결합
for file in csv_files:
    # CSV 파일을 읽습니다
    data = pd.read_csv(file)

    # 데이터를 필요한 열을 기준으로 그룹화하여 총 유동인구 수를 구합니다
    data_grouped = data.groupby(['DPR_CELL_ID', 'ARV_CELL_ID', 'P_YYYYMMDD']).agg({
        'DPR_MT10_UNIT_TM': 'first',
        'DPR_X_AXIS_WGS': 'first',
        'DPR_Y_AXIS_WGS': 'first',
        'ARV_MT10_UNIT_TM': 'first',
        'ARV_X_AXIS_WGS': 'first',
        'ARV_Y_AXIS_WGS': 'first',
        'POPL_CNT': 'sum'  # 유동인구 수 총합
    }).reset_index()

    # 필요한 열만 선택하여 새로운 데이터프레임에 추가
    output_columns = ['P_YYYYMMDD', 'DPR_MT10_UNIT_TM', 'DPR_CELL_ID', 'DPR_X_AXIS_WGS', 'DPR_Y_AXIS_WGS',
                      'ARV_MT10_UNIT_TM', 'ARV_CELL_ID', 'ARV_X_AXIS_WGS', 'ARV_Y_AXIS_WGS', 'POPL_CNT']
    processed_data = data_grouped[output_columns]

    # 결합된 데이터에 추가
    combined_data = pd.concat([combined_data, processed_data], ignore_index=True)

# 날짜별로 데이터를 나누어 CSV로 저장
unique_dates = combined_data['P_YYYYMMDD'].unique()  # P_YYYYMMDD 열을 기준으로 분리
for date in unique_dates:
    date_filtered_data = combined_data[combined_data['P_YYYYMMDD'] == date]
    output_filename = os.path.join(output_path, f'{date}.csv')  # 경로 포함 파일명 저장
    date_filtered_data.to_csv(output_filename, index=False)

    # 저장된 경로와 파일명을 출력
    print(f"파일이 저장되었습니다: {output_filename}")

# 결과 출력
print(f"전체 데이터를 처리하여 {len(unique_dates)}개의 파일로 분할 저장했습니다.")

## 일자별 -> 시간별 통합(1시간 단위)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Read the combined data (replace 'combined_output.csv' with actual combined file if needed)
data = pd.read_csv('/content/drive/MyDrive/여의도_불꽃축제_데이터/성별,연령,국적 통합 데이터/2023-10-07.csv')  # Replace with the actual file name if different

# 1. Convert DPR_MT10_UNIT_TM to 24-hour format with only time
data['DPR_MT10_UNIT_TM'] = pd.to_datetime(data['DPR_MT10_UNIT_TM'], errors='coerce').dt.strftime('%H:%M:%S')

# 2. Handle potential NaN values and convert to strings for further processing
data['DPR_MT10_UNIT_TM'] = data['DPR_MT10_UNIT_TM'].fillna('00:00:00')

# 3. Adjust DPR_MT10_UNIT_TM so that times are bucketed correctly (e.g., 23:00:01 to 23:59:59 -> 00:00:00)
def adjust_to_hour_bucket(time_str):
    """Adjusts a time string to represent the next appropriate hour bucket, wrapping around 00:00:00 for late-night entries."""
    time = datetime.strptime(time_str, '%H:%M:%S')
    # Special handling for times close to midnight
    if time.hour == 23 and (time.minute > 0 or time.second > 0):
        return '00:00:00'
    # Increment by 1 hour if not exactly at the hour
    if time.minute > 0 or time.second > 0:
        time += timedelta(hours=1)
    return time.replace(minute=0, second=0).strftime('%H:%M:%S')

# Apply the adjustment function to DPR_MT10_UNIT_TM
data['DPR_MT10_UNIT_TM'] = data['DPR_MT10_UNIT_TM'].apply(adjust_to_hour_bucket)

# 4. Group by 'DPR_CELL_ID', 'ARV_CELL_ID', 'P_YYYYMMDD', and 'DPR_MT10_UNIT_TM' and sum POPL_CNT
grouped_data = data.groupby(['DPR_CELL_ID', 'ARV_CELL_ID', 'P_YYYYMMDD', 'DPR_MT10_UNIT_TM'], as_index=False).agg({
    'POPL_CNT': 'sum',
    'DPR_X_AXIS_WGS': 'first',
    'DPR_Y_AXIS_WGS': 'first',
    'ARV_MT10_UNIT_TM': 'first',
    'ARV_X_AXIS_WGS': 'first',
    'ARV_Y_AXIS_WGS': 'first'
})

# 5. Split and save data based on P_YYYYMMDD and DPR_MT10_UNIT_TM
output_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/시간별 인구수 통합 데이터/"  # 저장할 폴더 경로

for dpr_time in grouped_data['DPR_MT10_UNIT_TM'].unique():
    subset = grouped_data[grouped_data['DPR_MT10_UNIT_TM'] == dpr_time]
    for date in subset['P_YYYYMMDD'].unique():
        filename = f"{date}_{dpr_time.replace(':', '')}.csv"
        date_subset = subset[subset['P_YYYYMMDD'] == date]

        # 지정된 경로로 파일 저장
        file_path = os.path.join(output_path, filename)
        date_subset.to_csv(file_path, index=False)
        print(f"파일이 저장되었습니다: {file_path}")